In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
X = pd.read_csv('https://raw.githubusercontent.com/zhukpm/da_spring2020/master/data/lw3/projects/1032172729/sample1.csv', header=None).to_numpy().ravel()
Y = pd.read_csv('https://raw.githubusercontent.com/zhukpm/da_spring2020/master/data/lw3/projects/1032172729/sample2.csv', header=None).to_numpy().ravel()
alpha = 0.05
m = X.size
n = Y.size
N = m + n
results = []

Проверим выборки на нормальность

In [3]:
T, p_val = stats.shapiro(X)
T, p_val

(0.8301942348480225, 4.97118506359584e-08)

Поскольку p уровень значимости < 0.05, то выборка Х ненормальная и следует использовать непараметрические критерии

Проверим на равенство средних при помощи манна-уитни (его реализовывать не надо, используем встроенный)

In [4]:
stat, p_val = stats.mannwhitneyu(X, Y, alternative='two-sided')
results.append(stat)
results.append(p_val)
if p_val > alpha:
    print('H0')
    results.append(0)
else:
    print('H1')
    results.append(1)

H1


Проверим на равенство средних при помощи ансари-бредли (реализуем ручками, без поправки на связки)

In [5]:
Z = np.r_[X, Y]
R = stats.rankdata(Z)
A = 0
for i in range(m):
    A += (N + 1) / 2 - np.abs(R[i] - (N + 1) / 2)
if N % 2 == 0:
    A_m = m * (N + 2) / 4
    A_v = m * n * (N + 2) * (N - 2) / (48 * (N - 1))
else:
    A_m = m * (N + 1) ** 2 / (4 * N)
    A_v = m * n * ((N ** 2) + 3) * (N + 1) / (48 * (N ** 2))
A_s = (A - A_m) / np.sqrt(A_v)
p_value = 2 * min(stats.norm.cdf(A_s), 1 - stats.norm.cdf(A_s))
results.append(A)
results.append(p_value)
if p_value > alpha:
    print('H0')
    results.append(0)
else:
    print('H1')
    results.append(1)

H1


In [6]:
results

[3602.0, 0.021208364245730117, 1, 3466.0, 0.002178574317640125, 1]

In [7]:
results = pd.DataFrame({'mt_s':results[0], 'mt_p':results[1], 'mt_h':results[2],
             'vt_s':results[3], 'vt_p':results[4], 'vt_h':results[5]}, index=[0])
results.to_csv('result.csv', index=False)

In [8]:
results

,mt_s,mt_p,mt_h,vt_s,vt_p,vt_h
0,3602.0,0.021208,1,3466.0,0.002179,1
